## Baixando mangás com Python

Alvo: centraldemangas.org
Mangá Inicial: One Piece
Ferramentas que vamos usar:

- editor de texto (sublime text3)
- terminal
- Python3
    - argparse
    - os
    - time
    - urllib.request
    - BeautifulSoup
    
### Porque fazer isso?
Bom, primeiro porque parece ser legal :)
Segundo porque essa que parece ser uma tarefa simples, pode acabar se tornando um pouco complexa, envolvendo alguns conceitos que podem ser desconhecidos numa primeira vez, mas que trazem um grande aprendizado.

O workflow é muito simples: vamos dizer para o programa qual o capítulo queremos baixar.
Agora vamos falar um pouco, mas bem rápido sobre as ferramentas que vamos usar.

### Standard libraries
- argparse
    - O módulo argparse é usado para criar CLIs de maneira fácil e amigável.
- os
    - Esse módulo fornece uma maneira portável de usar funcionalidade do sistema operacional.
- time
    - Funções realacionadas ao tempo.
- urllib.request
    - Esse módulo define funções e classes que ajudam a "abrir" e "ler" URLs (maioria HTTP).
- BeautifulSoup
    - É uma biblioteca escrita em Python projetada para parsear páginas.

Bom, vamos deixar de papo e começar a programWAIT! Antes disso precisamos fazer uma coisa.

### Preparar o ambiente
Iremos isolar nosso ambiente do resto do sistema, para podermos instalar todas as dependências do projeto de maneira segura.
````shell
$ mkvirtualenv -p python3 pymanga
$ pip install beautifulsoup4
````

Preparado nosso ambiente, e instalada nossas dependências, agora sim vamos iniciar a programação :)

### 1ª etapa - Função `main()`
````python
import argparse
import os
import time
import urllib.request

from bs4 import BeautifulSoup


def main():
    parser = argparse.ArgumentParser(
        prog='PyManga',
        description='An easily way to download your favorite mangas'
    )
    parser.add_argument(
        '-m', '--manga',
        type=str, help="Type the manga's name"
    )
    parser.add_argument(
        '-c', '--chapter',
        type=str, help='Type the chapter you wish'
    )

    args = parser.parse_args()

    if args.manga and args.chapter:
        download_chapter(args.chapter)
````

### 2ª etapa - Função `create_folder()`

````python
def create_folder(chapter):
    folder = 'capitulo-{}'.format(chapter)
    try:
        os.mkdir(folder)
        return folder
    except OSError:
        print('Diretório já existente')
        return folder
````

### 3ª etapa - Função `download_chapter()`
````python
BASE_URL_ONE_PIECE = 'http://mangaop.info/capitulos/{}#1'

def download_chapter(chapter):
    url = BASE_URL_ONE_PIECE.format(chapter)
    req = urllib.request.Request(url)
    content = urllib.request.urlopen(req).read()
````
O que nós fizemos até o momento, foi ajustar a nossa URL com o capítulo do mangá que o usuário quer baixar, usando o método `format()` da classe string. 
Depois disso, nós criamos um objeto `Request` usando a url como parâmetro. Após isso, usamos este objeto como parâmetro para a função `urlopen()` que de fato abre uma conexão e captura a página. O método `read()` retorna conteúdo deste objeto.

````python
    soup = BeautifulSoup(content, 'html.parser')
````
Tendo o html da página, agora nós criamos um objeto BeautifulSoup, usando o `html.parser` como parser. Agora, antes de seguir para o passo seguinte, vale dar uma olhada no html. Primeiramente nota-se que o HTML gerado no navegador é **muito** diferente do HTML que a requisição retorna. Um dos motivos é que essa requisição feita pelo Python, não tem como executar os códigos JS contidos na páginas, que depois de carregados podem e alteram o DOM da página. 

Sendo assim, tive que analisar o HTML para ver como faria para capturar o que eu quero. Após finalizar esta análise, pude notar que a URL da imagem do capítulo estava dentro de uma variável de um código JS, ou seja, dentro de uma tag `<script>`. Bom, aqui veio o pulo do gato:
````python
    words = soup.findAll('script')[16].string.split()
````
O método `findAll()` me retorna uma lista com todas as ocorrências da tag que eu passar como parâmetro. Após isso eu notei que a script que estava no índice 16 era o que continha a variável com a URL que eu queria. Então uso o atributo `string` que me retorna o toda a tag, inclusive seu conteúdo, como texto, e dei um `split()`, que retorna uma lista com as palavras naquela string, usando como separador o espaço entre as palavras.
````python
manga_url = ''
    for word in words:
        # check if word contains 'http' string
        if 'http' in word:
            # if so, assign it to manga_url and break the for
            manga_url = word
            break
````
Sendo assim, minha solução é: percorrer a lista e verificar se naquele item da lista há a string `http`. Se houver, este será a URL da imagem, uma vez que dentro deste script só há este único link. Porém, ele retorna o link como está no código, com aspas duplas (e.g. `"http://google.com"`).
````python
    manga_url = manga_url.replace('"', '')
````
Antes de continuar, vamos expôr algumas informações. O plano é baixar todas as imagens daquele capítulo. Eu pude notar que as imagens seguem um padrão. Digamos que estejamos com o capítulo 809 em questão, este padrão seria o seguinte:
* imagem 1: http://mangas2016.centraldemangas.com.br/one_piece/one_piece809-01.jpg
* imagem 2: http://mangas2016.centraldemangas.com.br/one_piece/one_piece809-02.jpg
Porém a URL que pegamos lá da tag `<script>`, é a seguinte:
* URL: http://mangas2016.centraldemangas.com.br/one_piece/one_piece809

Sendo assim, o que eu preciso fazer para pegar todas as imagens deste capítulo, é saber quantas páginas ela tem.
````python
    number_img = int(soup.find('select', id='capPages').text.split()[-1:][0])
````
Eu capturo a tag `<select>` que contém as opções com os números de imagens, uso o atributo `text` que me retorna o os `value` das tags `<option>` desse `<select>`, pego o último valor e o converto para `int`. Pronto, tenho a quantidade de imagens que esse capítulo tem. Agora vamos começar o processo de baixar as imagens.
````python
    folder = create_folder(chapter)
    for i in range(1, number_img+1):
        url = manga_url+'-{}.jpg'.format(i)
        filename = 'capitulo-{}.jpg'.format(i)

        print(url) # apenas para sabermos a URL que será usada
        req = urllib.request.Request(url)
        content = urllib.request.urlopen(req)
        
        with open(os.path.join(folder, filename), 'wb') as f:
            f.write(content.read())
            print('Arquivo salvo com sucesso! - {}'.format(filename))
    return
````
Nós percorremos com o for de 1 até a última imagem, e adicionamos ao final da imagem o número da imagem em questão, para completar a URL da imagem (como no exemplo lá de cima). Construída a URL, crio um `Request` mais uma vez, e uso no `urlopen()`. Após crio o arquivo que será salvo no caminho da pasta que criamos anteriormente (antes de iniciar o loop).

Pronto, teoricamente já finalizamos nossa função para baixar a imagem, e salvá-la como um arquivo na nossa máquina. Agora para começarmos a usar nosso script, basta um pequeno detalhe:
````python
    if __name__ == '__main__':
        main()
````
Dessa maneira, quando executarmos o script, ele vai chamar a função `main()` que é a responsável por pegar os valores passados pela linha de comando.

Vamos testar o programa! Vou baixar o capítulo 725

`python pymanga.py -m "One Piece" -c 725`

No terminal, você vai ver o seguinte:
````shell
http://mangas2013.centraldemangas.com.br/one_piece/one_piece725-1.jpg
Arquivo salvo com sucesso! - capitulo-1.jpg
http://mangas2013.centraldemangas.com.br/one_piece/one_piece725-2.jpg
Arquivo salvo com sucesso! - capitulo-2.jpg
http://mangas2013.centraldemangas.com.br/one_piece/one_piece725-3.jpg
Arquivo salvo com sucesso! - capitulo-3.jpg
http://mangas2013.centraldemangas.com.br/one_piece/one_piece725-4.jpg
Arquivo salvo com sucesso! - capitulo-4.jpg
http://mangas2013.centraldemangas.com.br/one_piece/one_piece725-5.jpg
Arquivo salvo com sucesso! - capitulo-5.jpg
http://mangas2013.centraldemangas.com.br/one_piece/one_piece725-6.jpg
Arquivo salvo com sucesso! - capitulo-6.jpg
http://mangas2013.centraldemangas.com.br/one_piece/one_piece725-7.jpg
Arquivo salvo com sucesso! - capitulo-7.jpg
http://mangas2013.centraldemangas.com.br/one_piece/one_piece725-8.jpg
Arquivo salvo com sucesso! - capitulo-8.jpg
http://mangas2013.centraldemangas.com.br/one_piece/one_piece725-9.jpg
Arquivo salvo com sucesso! - capitulo-9.jpg
http://mangas2013.centraldemangas.com.br/one_piece/one_piece725-10.jpg
Arquivo salvo com sucesso! - capitulo-10.jpg
http://mangas2013.centraldemangas.com.br/one_piece/one_piece725-11.jpg
Arquivo salvo com sucesso! - capitulo-11.jpg
http://mangas2013.centraldemangas.com.br/one_piece/one_piece725-12.jpg
Arquivo salvo com sucesso! - capitulo-12.jpg
http://mangas2013.centraldemangas.com.br/one_piece/one_piece725-13.jpg
Arquivo salvo com sucesso! - capitulo-13.jpg
http://mangas2013.centraldemangas.com.br/one_piece/one_piece725-14.jpg
Arquivo salvo com sucesso! - capitulo-14.jpg
http://mangas2013.centraldemangas.com.br/one_piece/one_piece725-15.jpg
Arquivo salvo com sucesso! - capitulo-15.jpg
````

Você vai notar que no mesmo diretório onde você salvou o arquivo pymanga.py, estará uma pasta `capitulo-725`, que é onde as imagens estarão salvas. Vamos visualizá-las...

Epa, parece que algo deu errado, não foi? Bom, a primeira coisa que estranhei a primeira vez foi o tamanho da imagem. Baixando ela pelo navegador, fica uns `300kB`, mas ali temos várias imagens com `39,1kB`. E ao tentar visualizar a imagem, o visualizador padrão do seu sistema não vai mostrar imagem nenhuma. O que deu errado? Se você for no site e pegar a URL da imagem, vai ver que as imagens iniciais tem a URL com final `one_piece725-01.jpg` ao invés de `one_piece725-1.jpg`. Ou seja, o que acontece é que estamos requisitando uma imagem que não existe, e quando não existe, ele redireciona para a home centraldemangas.org, baixa esse HTML e salva como imagem.

### Tratando das possibilidades dos erros
Enquanto eu desenvolvia este sistema, me deparei com diversas situações e problemas. Vou listar alguns deles:

- Redirect quando o arquivo não existe
- Final da URL errada
- Conexão negada por causa do header
- Conexão negada por múltiplas requisições do mesmo IP numa faixa de tempo
- host errado para o servidor

Aquele arquivo nada mais é do que: uma página HTML. Se você renomear o final da extensão de `.jpg` para `.html`, poderá ver seu conteúdo, que será a home do site centraldemangas.org.

Como tratar esse problema?

### Criando nosso arquivo header
````python
# arquivo config.py
HEADER_ONE_PIECE = {
    'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64; rv:42.0) Gecko/20100101 \
                   Firefox/42.0',
    'Host': 'mangas2014.centraldemangas.com.br',
    'Accept': 'image/png,image/*;q=0.8,*/*;q=0.5',
    'Accept-Language': 'pt-BR,en;q=0.5',
    'Referer': 'http://mangaop.info/capitulos/747'
}
````

### Otimizando nosso código
````python
# o import abaixo deve ir para o início do arquivo
from configs import HEADER_ONE_PIECE

    folder = create_folder(chapter)
    for i in range(1, number_img+1):
        time.sleep(3) # aguarda 3s entre cada requisição

        print(url) # apenas para sabermos a URL que será usada
        req = urllib.request.Request(url)
        content = urllib.request.urlopen(req)
        
        # Esse é o pulo do gato para evitar o final da URL errada
        if i in range(1, 10):
            final = '0'+str(i)
            url = manga_url+'-{}.jpg'.format(final)
            filename = 'capitulo-{}.jpg'.format(final)
        else:
            url = manga_url+'-{}.jpg'.format(i)
            filename = 'capitulo-{}.jpg'.format(i)
        
        try:
            # abaixo faremos uso da variável com infos do header para esse site
            regex = re.compile('(^http:\/\/.*\.centraldemangas\.com\.br)')
            host = regex.search(url).group()
            HEADER_ONE_PIECE.update(host=host)
            print('host:', HEADER_ONE_PIECE['host'])
            req = urllib.request.Request(url, headers=HEADER_ONE_PIECE)
            # print(url) ativar esse print caso queira ver a URL da imagem no terminal
            content = urllib.request.urlopen(req)
        except urllib.error.HTTPError:
            print('Arquivo {} indisponível'.format(filename))
            continue
        
        with open(os.path.join(folder, filename), 'wb') as f:
            f.write(content.read())
            print('Arquivo salvo com sucesso! - {}'.format(filename))
    return
````

##### to-do explicar o que se passa no código acima